In [2]:
from utils import *

def test():

    # Load model
    model = torch.jit.load('model.pt')
    
    # import input from database
    r_seq = db2Rseq()
    h_seq = db2Hseq(h_step = 4)
    t = dt2T()
    s_attrs, _ = db2S()

    # model inference
    minutes = [20, 40, 60, 120]

    for i, m in enumerate(minutes):
        globals()[f'res_{m}'] = model(r_seq.float(), h_seq[:, i, :, :].float(), t[:, i, :].int(), s_attrs.float())

    # export output to database
    outputs = torch.hstack([res_20, res_40, res_60, res_120])

    try:
        y2db(outputs) 
        return {'time':get_now(), 'status':'Success'}

    except: return {'time': get_now(), 'status':'Failed'}


In [3]:
test()

{'time': '2023-08-21 13:00:00', 'status': 'Success'}

### Resolve encoding error in Model

In [3]:
import torch
from conti_model import MultiSeqBase

# load model
model = MultiSeqBase(hidden_size = 16, embedding_dim = 4, dropout_p = 0.2)
optim = torch.optim.Adam(model.parameters(), weight_decay=1e-3)

checkpoint = torch.load('test_model_and_optimizer_conti.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])

In [4]:
model

MultiSeqBase(
  (lstm_r1): LSTM(1, 16, batch_first=True)
  (fc_r1): Linear(in_features=16, out_features=16, bias=True)
  (lstm_h1): LSTM(1, 16, batch_first=True)
  (fc_h1): Linear(in_features=16, out_features=16, bias=True)
  (timeslot_embedding): Embedding(144, 4)
  (dow_embedding): Embedding(7, 4)
  (we_embedding): Embedding(2, 4)
  (fc_b1): Linear(in_features=12, out_features=128, bias=True)
  (fc_b2): Linear(in_features=128, out_features=64, bias=True)
  (fc_b3): Linear(in_features=64, out_features=64, bias=True)
  (fc_cat): Linear(in_features=96, out_features=64, bias=True)
  (top): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [17]:
scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, 'model.pt')

In [1]:
import torch
model = torch.jit.load('model.pt')